import the necessary library

In [22]:
import pandas as pd
# !pip install psycopg2
import pandas as pd
import psycopg2
from psycopg2 import Error
from sqlalchemy import create_engine


In [3]:
# Provide the file paths to the downloaded CSV files
borrower_csv_path = 'Borrower_Data - Sheet1.csv'
loan_csv_path = 'Loan_Data - Sheet1 - Loan_Data - Sheet1.csv'
payment_schedule_csv_path = 'Schedule_Data.xlsx - Sheet1.csv'
repayment_csv_path = 'Repayment_Data - Sheet1 - Repayment_Data - Sheet1 (1).csv'

# Read the CSV files into pandas DataFrames
borrower_df = pd.read_csv(borrower_csv_path)
loan_df = pd.read_csv(loan_csv_path)
payment_schedule_df = pd.read_csv(payment_schedule_csv_path)
repayment_df = pd.read_csv(repayment_csv_path)


In [7]:
repayment_df['Amount_paid'] = pd.to_numeric(repayment_df['Amount_paid'], errors='coerce')


In [8]:
repayment_df['amount_at_risk'] = repayment_df.groupby('loan_id(fk)')['Amount_paid'].cumsum()

In [9]:
# Calculate PAR Days
repayment_df['Date_paid'] = pd.to_datetime(repayment_df['Date_paid'])
payment_schedule_df['Expected_payment_date'] = pd.to_datetime(payment_schedule_df['Expected_payment_date'])
repayment_df['PAR_Days'] = (repayment_df['Date_paid'] - payment_schedule_df['Expected_payment_date']).dt.days

# Calculate amount_at_risk
repayment_df['amount_at_risk'] = repayment_df.groupby('loan_id(fk)')['Amount_paid'].cumsum()

# Generate the SQL query using the desired columns
query = '''
SELECT borrower.Borrower_id, borrower.State, borrower.City, borrower.zip_code,
       loan.Loan_id, loan.Term, loan.InterestRate, loan.LoanAmount, loan.Downpayment,
       payment_schedule.Expected_payment_date, payment_schedule.Expected_payment_amount,
       loan_payment.PAR_Days, loan_payment.amount_at_risk
FROM Borrower_table AS borrower
JOIN Loan_table AS loan ON borrower.Borrower_id = loan.Borrower_id
JOIN Payment_Schedule AS payment_schedule ON loan.Loan_id = payment_schedule.loan_id
JOIN Loan_payment AS loan_payment ON loan.Loan_id = loan_payment.loan_id
ORDER BY borrower.Borrower_id, loan.Loan_id, payment_schedule.Expected_payment_date
'''

# Print the SQL query
print(query)



SELECT borrower.Borrower_id, borrower.State, borrower.City, borrower.zip_code,
       loan.Loan_id, loan.Term, loan.InterestRate, loan.LoanAmount, loan.Downpayment,
       payment_schedule.Expected_payment_date, payment_schedule.Expected_payment_amount,
       loan_payment.PAR_Days, loan_payment.amount_at_risk
FROM Borrower_table AS borrower
JOIN Loan_table AS loan ON borrower.Borrower_id = loan.Borrower_id
JOIN Payment_Schedule AS payment_schedule ON loan.Loan_id = payment_schedule.loan_id
JOIN Loan_payment AS loan_payment ON loan.Loan_id = loan_payment.loan_id
ORDER BY borrower.Borrower_id, loan.Loan_id, payment_schedule.Expected_payment_date



In [5]:
print(repayment_df.columns)


Index(['loan_id(fk)', 'payment_id(pk)', 'Amount_paid', 'Date_paid',
       'PAR_Days'],
      dtype='object')


In [18]:
try:
    # Replace the connection details with your own
    connection = psycopg2.connect(
        host="localhost",
        database="autochek_db",
        user="auto",
        password="auto"
    )
    cursor = connection.cursor()
    print("Successfully connected to the PostgreSQL database")

except (Exception, Error) as e:
    print("Error while connecting to the PostgreSQL database:", e)


Successfully connected to the PostgreSQL database


In [61]:
# SQL statements to create the tables
create_table_queries = [
    '''
    CREATE TABLE IF NOT EXISTS Borrower_table (
        Borrower_id INTEGER PRIMARY KEY,
        State TEXT,
        City TEXT,
        zip_code TEXT
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Loan_table (
        Borrower_id INTEGER REFERENCES Borrower_table (Borrower_id),
        Loan_id INTEGER PRIMARY KEY,
        Date_of_release DATE,
        Term INTEGER,
        InterestRate FLOAT,
        LoanAmount FLOAT,
        Downpayment FLOAT,
        Payment_frequency TEXT,
        Maturity_date DATE
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Schedule_Table (
        loan_id INTEGER REFERENCES Loan_table (Loan_id),
        schedule_id INTEGER PRIMARY KEY,
        Expected_payment_date DATE,
        Expected_payment_amount FLOAT
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Repayment_Table (
        loan_id INTEGER REFERENCES Loan_table (Loan_id),
        payment_id INTEGER PRIMARY KEY,
        Amount_paid FLOAT,
        Date_paid DATE,
        PAR_Days INTEGER,
        amount_at_risk FLOAT
    );
    '''
]

try:
    # Execute the create table queries
    for query in create_table_queries:
        cursor.execute(query)
    connection.commit()
    print("Tables created successfully")

except (Exception, Error) as e:
    print("Error while creating tables:", e)


Tables created successfully


In [62]:
# Create a SQLAlchemy engine using the connection
engine = create_engine('postgresql+psycopg2://', creator=lambda: connection)

# Use the engine to load data into tables
try:
    
    borrower_df.to_sql('Borrower_table', con=engine, if_exists='replace', index=False)
    loan_df.to_sql('Loan_table', con=engine, if_exists='replace', index=False)
    payment_schedule_df.to_sql('Schedule_Table', con=engine, if_exists='replace', index=False)
    repayment_df.to_sql('Repayment_Table', con=engine, if_exists='replace', index=False)
    
    print("Data loaded into the tables successfully")

except (Exception, Error) as e:
    print("Error while loading data into tables:", e)

Data loaded into the tables successfully


In [29]:
borrower_df

,Borrower_Id,State,City,zip code,borrower_credit_score
0,123fd35,dfgc,12olki,19473,4
1,123fd36,rtgio,89hfg,923475,a


In [30]:
loan_df

,Borrower_id,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date
0,123fd35,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,1.007903e+05,01/31/2022
1,123fd36,9190i0-nbfb,02/01/2021,24,1.05,12394031,1000000,5.164180e+05,02/29/2023
2,123fd35,09u924rbwf,04/16/2021,36,1.05,45784937,1200000,1.271804e+06,04/30/2024
3,123fd36,3240-9rfwb,03/26/2021,12,1.05,1234890,132000,1.029075e+05,03/31/2022


In [31]:
payment_schedule_df

,loan_id,schedule_id,Expected_payment_date,Expected_payment_amount
0,32u09wekjbfje,3434r409km123456,2021-02-27,100790.3333
1,32u09wekjbfje,3434r409km123457,2021-03-27,100790.3333
2,32u09wekjbfje,3434r409km123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409km123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409km123460,2021-06-27,100790.3333
...,...,...,...,...
79,3240-9rfwb,klnin09u4nj33471,2021-11-26,102907.5000
80,3240-9rfwb,klnin09u4nj33472,2021-12-26,102907.5000
81,3240-9rfwb,klnin09u4nj33473,2022-01-26,102907.5000
82,3240-9rfwb,klnin09u4nj33474,2022-02-26,102907.5000


In [32]:
repayment_df

,loan_id(fk),payment_id(pk),Amount_paid,Date_paid,PAR_Days,amount_at_risk
0,32u09wekjbfje,3434r409kmPAID123456,NaN,1970-01-01 00:00:00.000100790,-18685,NaN
1,32u09wekjbfje,3434r409kmPAID123457,NaN,1970-01-01 00:00:00.000100790,-18713,NaN
2,32u09wekjbfje,3434r409kmPAID123458,NaN,1970-01-01 00:00:00.000100790,-18744,NaN
3,32u09wekjbfje,3434r409kmPAID123459,NaN,1970-01-01 00:00:00.000100790,-18774,NaN
4,32u09wekjbfje,3434r409kmPAID123460,NaN,1970-01-01 00:00:00.000100790,-18805,NaN
...,...,...,...,...,...,...
79,3240-9rfwb,klnin09u4njPAID33471,NaN,1970-01-01 00:00:00.000102907,-18957,NaN
80,3240-9rfwb,klnin09u4njPAID33472,NaN,1970-01-01 00:00:00.000102907,-18987,NaN
81,3240-9rfwb,klnin09u4njPAID33473,NaN,1970-01-01 00:00:00.000102907,-19018,NaN
82,3240-9rfwb,klnin09u4njPAID33474,NaN,1970-01-01 00:00:00.000102907,-19049,NaN


In [60]:
try:
    # Replace the connection details with your own
    connection = psycopg2.connect(
        host="localhost",
        database="autochek_db",
        user="auto",
        password="auto"
    )
    cursor = connection.cursor()
    print("Successfully connected to the PostgreSQL database")

except (Exception, Error) as e:
    print("Error while connecting to the PostgreSQL database:", e)


Successfully connected to the PostgreSQL database


In [64]:
# Generate the SQL query using the desired columns
query = '''
SELECT borrower.Borrower_id, borrower.State, borrower.City, borrower.zip_code,
       loan.Loan_id, loan.Term, loan.InterestRate, loan.LoanAmount, loan.Downpayment,
       payment_schedule.Expected_payment_date, payment_schedule.Expected_payment_amount,
       loan_payment.PAR_Days, loan_payment.amount_at_risk
FROM Borrower_table AS borrower
JOIN Loan_table AS loan ON borrower.Borrower_id = loan.Borrower_id
JOIN Schedule_Table AS payment_schedule ON loan.Loan_id = payment_schedule.loan_id
JOIN Repayment_Table AS loan_payment ON loan.Loan_id = loan_payment.loan_id
ORDER BY borrower.Borrower_id, loan.Loan_id, payment_schedule.Expected_payment_date
'''

try:
    # Execute the SQL query
    cursor.execute(query)
    result = cursor.fetchall()

    # Print the query results
    for row in result:
        print(row)

except (Exception, Error) as e:
    print("Error while executing the SQL query:", e)

Error while executing the SQL query: current transaction is aborted, commands ignored until end of transaction block



In [41]:
from parse import parse

ModuleNotFoundError: No module named 'parse'

In [42]:
pip install parse

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [43]:
from parse import parse

In [46]:
borrower_df = parse.change_column_dtype(borrower_df)
loan_df = parse.change_column_dtype(loan_df)
schedule_df = parse.change_column_dtype(schedule_df)
repayment__df = parse.change_column_dtype(repayment_df)

AttributeError: 'function' object has no attribute 'change_column_dtype'

In [45]:
def change_column_dtype(df, column_dtype_mapping):
    for column, dtype in column_dtype_mapping.items():
        df[column] = df[column].astype(dtype)
    return df

In [51]:
# Define the desired column data types
import datetime
column_dtype_mapping = {
    'loan_id': str,
    'schedule_id': str,
    'Expected_payment_date':datetime.datetime ,
    'Expected_payment_amount': float,
}

# Apply the data type changes to the DataFrame
payment_schedule_df = change_column_dtype(payment_schedule_df, column_dtype_mapping)

TypeError: dtype '<class 'datetime.datetime'>' not understood

In [53]:
payment_schedule_df['Expected_payment_date'] = pd.to_datetime(payment_schedule_df['Expected_payment_date'], errors='coerce')

In [54]:
repayment_df['Amount_paid'] = pd.to_datetime(repayment_df['Amount_paid'], errors='coerce')


In [55]:
loan_df['Date_of_release'] = pd.to_datetime(loan_df['Date_of_release'], errors='coerce')
loan_df['Maturity_date'] = pd.to_datetime(loan_df['Maturity_date'], errors='coerce')

In [56]:
payment_schedule_df.head()

,loan_id,schedule_id,Expected_payment_date,Expected_payment_amount
0,32u09wekjbfje,3434r409km123456,2021-02-27,100790.3333
1,32u09wekjbfje,3434r409km123457,2021-03-27,100790.3333
2,32u09wekjbfje,3434r409km123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409km123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409km123460,2021-06-27,100790.3333


In [57]:
repayment_df.rename(columns={'Amount_paid':'date_paid'},inplace=True)
repayment_df.rename(columns={'Date_paid':'Amount_paid'},inplace=True)